## File containing funcitons for extracting data from dataset

In [1]:
import numpy as np
import scipy
from scipy.io import loadmat
import mne, glob
import pandas as pd
import xlrd

In [2]:
#Function that takes in a path for a spreadsheet that contains the channel names and returns the data extracted from the spreadsheet as a list.
#Assumes that the channel names are in the first column
def extarctChannelNames(channelFile):
    channelNames=channelFile.iloc[:,0]
    channelNames=np.ndarray.tolist(pd.Series.to_numpy(channelNames))
    channelNames.insert(0,"FP1")
    return channelNames

In [3]:
#Function takes in a list of files (in our case that would be .mat files) and sorts them, it would assume that there are 16 files and the 16th file
#will be the file that contains the labels.  Returns the list of files without the label file and the label  file itself
def getLabelsAndMatfiles(matfiles):
    matfiles.sort()
    labels=loadmat(matfiles[15])
    matfiles.pop()
    label_list=np.ndarray.tolist(labels['label'])[0]
    return matfiles, label_list

In [4]:
# function that extracts all the data from the given dataset in the file 'Preprocessed_EEG' and returns a dictionary with each entry being the raw data gathred 
# from a given file
def getRawArrayData():
    channelNames=extarctChannelNames(pd.read_excel("Preprocessed_EEG/channel-order.xlsx"))
    matfiles,label_list = getLabelsAndMatfiles(glob.glob('Preprocessed_EEG/*.mat'))
    info=mne.create_info(channelNames,200,'eeg')
    subjects={}
    indexs=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
    for x in range(0,15):
        clip_info={}
        used_data = loadmat(matfiles[x])
        del used_data["__header__"]
        del used_data["__version__"]
        del used_data["__globals__"]
        keyName=list(used_data.keys())[0][:-1]
        for ind,i in enumerate(indexs):
            rawData=used_data[keyName + str(i)]
            clip_info[ind] = mne.io.RawArray(rawData,info,verbose=0)
        subjects[x]=clip_info
    return subjects